In [1]:
import os
os.chdir(os.path.pardir)
from dataset.dataset import Dataset
from evaluation_metrics.diversity_metrics import Topic_diversity
from evaluation_metrics.topic_significance_metrics import KL_uniform
from skopt import gp_minimize, forest_minimize, dummy_minimize
from optimization.optimizer import Optimizer
from skopt.space.space import Real, Integer
import multiprocessing as mp
from models.ETM import main
import torch
import numpy as np

In [2]:
dataset = Dataset()
dataset.load("preprocessed_datasets/newsgroup/newsgroup_lemmatized_10")

True

In [3]:
# Load model
model = main.ETM_Wrapper()

In [4]:
model.hyperparameters['epochs'] = 5

In [5]:
model.train_model(dataset, model.hyperparameters)




model: ETM(
  (t_drop): Dropout(p=0.0, inplace=False)
  (theta_act): ReLU()
  (rho): Linear(in_features=300, out_features=1268, bias=False)
  (alphas): Linear(in_features=300, out_features=10, bias=False)
  (q_theta): Sequential(
    (0): Linear(in_features=1268, out_features=800, bias=True)
    (1): ReLU()
    (2): Linear(in_features=800, out_features=800, bias=True)
    (3): ReLU()
  )
  (mu_q_theta): Linear(in_features=800, out_features=10, bias=True)
  (logsigma_q_theta): Linear(in_features=800, out_features=10, bias=True)
)
****************************************************************************************************
Epoch----->1 .. LR: 0.005 .. KL_theta: 0.04 .. Rec_loss: 502.38 .. NELBO: 502.42
****************************************************************************************************
****************************************************************************************************
Epoch----->2 .. LR: 0.005 .. KL_theta: 0.62 .. Rec_loss: 497.39 .. NELBO: 498.01


{'topics': [['victim',
   'method',
   'past',
   'card',
   'generation',
   'library',
   'playoff',
   'fire',
   'search'],
  ['school',
   'card',
   'encryption',
   'victim',
   'license',
   'uunet',
   'base',
   'window',
   'explanation'],
  ['turn',
   'playoff',
   'thread',
   'victim',
   'library',
   'be',
   'guess',
   'basis',
   'attempt'],
  ['turn',
   'playoff',
   'thread',
   'victim',
   'library',
   'be',
   'guess',
   'basis',
   'attempt'],
  ['card',
   'generation',
   'victim',
   'past',
   'technical',
   'license',
   'uunet',
   'method',
   'library'],
  ['victim',
   'playoff',
   'library',
   'license',
   'school',
   'guess',
   'turn',
   'card',
   'past'],
  ['security',
   'shipping',
   'scale',
   'son',
   'mostly',
   'screw',
   'back',
   'fire',
   'time'],
  ['generation',
   'card',
   'victim',
   'method',
   'past',
   'hope',
   'technical',
   'military',
   'search'],
  ['turn',
   'playoff',
   'victim',
   'thread',
   '